In [10]:
import numpy as np
import intracluster_smote
from intracluster_smote import IntraclusterSmote
import evaluation
import matplotlib.pyplot as plt
import mnist_utils
import pandas as pd
from IPython.display import display
import sklearn
import sklearn.cluster

In [63]:
# Import Ecoli Dataset
ecoli_data_raw = pd.read_csv('datasets/ecoli.data.txt', delim_whitespace=True, header=None, names=['Sequence Name','mcg','gvh','lip','chg','aac','alm1','alm2','class'])
ecoli_data_target = ecoli_data_raw['class'].as_matrix()
ecoli_data = ecoli_data_raw.drop(['Sequence Name','class'], axis=1).as_matrix()
ecoli_scaler = sklearn.preprocessing.MinMaxScaler()
ecoli_data = ecoli_scaler.fit_transform(ecoli_data)
ecoli_train, ecoli_validate, ecoli_train_target, ecoli_validate_target = sklearn.model_selection.train_test_split(ecoli_data, ecoli_data_target)

In [4]:
datasets = {
    'MNIST Imbalanced 7+8': {
        'Data': mnist_utils.load_binary_imbalanced(classes=[7,8], ratio=0.2),
        'Imbalance Ratio': 1/5,
        'Minority Class': 8,
        'Scaler': sklearn.preprocessing.MinMaxScaler().fit([[0],[255]]),
        'k': 2
    },
    'MNIST Imbalanced 1+7': {
        'Data': mnist_utils.load_binary_imbalanced(classes=[1,7], ratio=0.2),
        'Imbalance Ratio': 1/5,
        'Minority Class': 7,
        'Scaler': sklearn.preprocessing.MinMaxScaler().fit([[0],[255]]),
        'k':2
    },
    'Ecoli': {
        'Data': ((ecoli_train, ecoli_train_target),(ecoli_validate, ecoli_validate_target)),
        'Imbalance Ratio': 1/5.46,
        'Minority Class':'pp',
        'Scaler': ecoli_scaler,
        'k':2
    }
}

In [5]:
clustering_methods = {
    'No Clustering': lambda X, k: np.zeros(X.shape[0],),
    'Random Clustering': lambda X, k: np.random.choice(k, size=(X.shape[0])),
    'K-Means': lambda X, k: sklearn.cluster.KMeans(n_clusters=k).fit(X).labels_
}

In [59]:
classifiers = {
    'Logistic Regression': sklearn.linear_model.LogisticRegression(),
    'Random Forest': sklearn.ensemble.GradientBoostingClassifier(),
    'Gradient Boosting': sklearn.ensemble.RandomForestClassifier(),
    'Support Vector': sklearn.svm.SVC()
}

In [62]:
imp.reload(evaluation)
classification_evaulation = {}
for dataset_name, dataset in datasets.items():
#     if dataset_name != 'Ecoli':continue
    (train_set, train_set_target), (validation_set, validation_set_target) = dataset['Data']
    for clustering_method_name, clustering_method in clustering_methods.items():
        #upsample
        oversampler = IntraclusterSmote( IntraclusterSmote.compute_synthetic_count(train_set.shape[0], dataset['Imbalance Ratio']) )
        oversampled_train_set, oversampled_train_set_target = oversampler.fit(train_set, train_set_target, (train_set_target == dataset['Minority Class']), clustering_method(train_set, dataset['k']))
        for classifier_name, classifier in classifiers.items():
            # classify
            classifier.fit(train_set,train_set_target)
            prediction = classifier.predict(validation_set)
            #evaluate
            classification_evaulation[(dataset_name,clustering_method_name,classifier_name)] = evaluation.evaluate_classification(prediction, validation_set_target)

display(pd.DataFrame(classification_evaulation).transpose())

C:\Users\D059331\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\D059331\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\D059331\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\D059331\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', a

Balanced F-Measure  \
Ecoli                K-Means           Gradient Boosting              0.562403   
                                       Logistic Regression            0.409572   
                                       Random Forest                  0.552394   
                                       Support Vector                 0.347098   
                     No Clustering     Gradient Boosting              0.660801   
                                       Logistic Regression            0.409572   
                                       Random Forest                  0.552394   
                                       Support Vector                 0.347098   
                     Random Clustering Gradient Boosting              0.569703   
                                       Logistic Regression            0.409572   
                                       Random Forest                  0.552394   
                                       Support Vector                 0.347098   
MNIST Imbalanced 1+7 K-Means           Gradient Boosting              0.991985   
                                       Logistic Regression            0.996014   
                                       Random Forest                  0.993333   
                                       Support Vector                 0.989275   
                     No Clustering     Gradient Boosting              0.991985   
                                       Logistic Regression            0.996014   
                                       Random Forest                  0.992014   
                                       Support Vector                 0.989275   
                     Random Clustering Gradient Boosting              0.987913   
                                       Logistic Regression            0.996014   
                                       Random Forest                  0.993333   
                                       Support Vector                 0.989275   
MNIST Imbalanced 7+8 K-Means           Gradient Boosting              0.965614   
                                       Logistic Regression            0.985140   
                                       Random Forest                  0.970279   
                                       Support Vector                 0.977754   
                     No Clustering     Gradient Boosting              0.965333   
                                       Logistic Regression            0.985140   
                                       Random Forest                  0.970279   
                                       Support Vector                 0.977754   
                     Random Clustering Gradient Boosting              0.969917   
                                       Logistic Regression            0.985140   
                                       Random Forest                  0.970279   
                                       Support Vector                 0.977754   

                                                            G-Measure / Fowlkes-Mallows Score  \
Ecoli                K-Means           Gradient Boosting                             0.877137   
                                       Logistic Regression                           0.851191   
                                       Random Forest                                 0.839906   
                                       Support Vector                                0.885219   
                     No Clustering     Gradient Boosting                             0.891532   
                                       Logistic Regression                           0.851191   
                                       Random Forest                                 0.839906   
                                       Support Vector                                0.885219   
                     Random Clustering Gradient Boosting                             0.859867   
                                       Logistic Regress